In [1]:
!pip install -r /mnt/c/Users/santhosh/Downloads/MedMamba-main/MedMamba-main/requirements.txt


Defaulting to user installation because normal site-packages is not writeable


In [14]:
!git clone https://github.com/YubiaoYue/MedMamba.git

Cloning into 'MedMamba'...
remote: Enumerating objects: 127, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 127 (delta 53), reused 90 (delta 28), pack-reused 4 (from 1)
Receiving objects: 100% (127/127), 353.24 KiB | 2.44 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [19]:
import os
os.chdir('/mnt/c/Users/santhosh/Downloads/MedMamba-main/MedMamba-main/MedMambagit') 

In [20]:
import sys

sys.path.append('/mnt/c/Users/santhosh/Downloads/MedMamba-main/MedMamba-main/MedMambagit')

In [21]:
import MedMamba

In [19]:
import os
import sys
import json

import torch
import torch.nn as nn
from torchvision import transforms, datasets
import torch.optim as optim
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np 
import importlib
import MedMamba

importlib.reload(MedMamba)


def calculate_metrics(y_true, y_pred):
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    return precision, recall, f1


def main():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("using {} device.".format(device))

    data_transform = {
        "train": transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(15),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]),
        "val": transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])}

    train_dataset = datasets.ImageFolder(root="/mnt/c/Users/santhosh/Downloads/MedMamba-main/MedMamba-main/colored_images",
                                         transform=data_transform["train"])
    train_num = len(train_dataset)

    flower_list = train_dataset.class_to_idx
    cla_dict = dict((val, key) for key, val in flower_list.items())
    # write dict into json file
    json_str = json.dumps(cla_dict, indent=4)
    with open('class_indices.json', 'w') as json_file:
        json_file.write(json_str)

    batch_size = 32
    nw = min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8])  # number of workers
    print('Using {} dataloader workers every process'.format(nw))

    train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size, shuffle=True,
                                               num_workers=nw)

    validate_dataset = datasets.ImageFolder(root="/mnt/c/Users/santhosh/Downloads/MedMamba-main/MedMamba-main/colored_images",
                                            transform=data_transform["val"])
    val_num = len(validate_dataset)
    validate_loader = torch.utils.data.DataLoader(validate_dataset,
                                                  batch_size=batch_size, shuffle=False,
                                                  num_workers=nw)
    print("using {} images for training, {} images for validation.".format(train_num,
                                                                           val_num))

    num_classes = len(cla_dict)
    model_name = "Medmamba"

    net = MedMamba.VSSM(num_classes=num_classes)
    net.to(device)

    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=0.0001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    epochs = 100
    best_acc = 0.0
    save_path = './{}Net.pth'.format(model_name)
    train_steps = len(train_loader)
    patience = 5
    early_stop_counter = 0

    for epoch in range(epochs):
        # Train
        net.train()
        running_loss = 0.0
        train_bar = tqdm(train_loader, file=sys.stdout)
        for step, data in enumerate(train_bar):
            images, labels = data
            optimizer.zero_grad()
            outputs = net(images.to(device))
            loss = loss_function(outputs, labels.to(device))
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch + 1, epochs, loss)

        # Validate
        net.eval()
        acc = 0.0  # accurate number / epoch
        val_loss = 0.0
        all_labels = []
        all_preds = []

        with torch.no_grad():
            val_bar = tqdm(validate_loader, file=sys.stdout)
            for val_data in val_bar:
                val_images, val_labels = val_data
                outputs = net(val_images.to(device))
                loss = loss_function(outputs, val_labels.to(device))
                val_loss += loss.item()
                
                predict_y = torch.max(outputs, dim=1)[1]
                acc += torch.eq(predict_y, val_labels.to(device)).sum().item()

                all_labels.extend(val_labels.cpu().numpy())
                all_preds.extend(predict_y.cpu().numpy())

        val_accurate = acc / val_num
        val_loss /= len(validate_loader)
        precision, recall, f1 = calculate_metrics(np.array(all_labels), np.array(all_preds))

        print('[epoch %d] train_loss: %.3f  val_loss: %.3f  val_accuracy: %.3f  precision: %.3f  recall: %.3f  f1: %.3f' %
              (epoch + 1, running_loss / train_steps, val_loss, val_accurate, precision, recall, f1))

        scheduler.step()

        if val_accurate > best_acc:
            best_acc = val_accurate
            torch.save(net.state_dict(), save_path)
            early_stop_counter = 0
        else:
            early_stop_counter += 1

        if early_stop_counter >= patience:
            print("Early stopping triggered. Stopping training.")
            break

    print('Finished Training')


if __name__ == '__main__':
    main()


torch.Size([1, 6])
using cuda:0 device.
Using 8 dataloader workers every process
using 3656 images for training, 3656 images for validation.
100%|██████████| 115/115 [00:26<00:00,  4.29it/s]
[epoch 1] train_loss: 1.309  val_loss: 1.158  val_accuracy: 0.520  precision: 0.408  recall: 0.520  f1: 0.388


/home/santhosh/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


100%|██████████| 115/115 [00:26<00:00,  4.33it/s]
[epoch 2] train_loss: 1.204  val_loss: 0.999  val_accuracy: 0.647  precision: 0.519  recall: 0.647  f1: 0.567


/home/santhosh/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


100%|██████████| 115/115 [00:26<00:00,  4.35it/s]
[epoch 3] train_loss: 1.116  val_loss: 0.954  val_accuracy: 0.675  precision: 0.520  recall: 0.675  f1: 0.587


...

/home/santhosh/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



[epoch 28] train_loss: 0.885  val_loss: 0.821  val_accuracy: 0.722  precision: 0.583  recall: 0.722  f1: 0.639
100%|██████████| 115/115 [00:29<00:00,  3.92it/s]

/home/santhosh/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



[epoch 29] train_loss: 0.894  val_loss: 0.817  val_accuracy: 0.722  precision: 0.586  recall: 0.722  f1: 0.640
100%|██████████| 115/115 [00:26<00:00,  4.31it/s]
[epoch 30] train_loss: 0.878  val_loss: 0.820  val_accuracy: 0.722  precision: 0.583  recall: 0.722  f1: 0.639
Early stopping triggered. Stopping training.
Finished Training


/home/santhosh/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import os
import sys
import json

import torch
import torch.nn as nn
from torchvision import transforms, datasets
import torch.optim as optim
from tqdm import tqdm
import importlib
import MedMamba  # import model
importlib.reload(MedMamba)

def main():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("using {} device.".format(device))

    data_transform = {
        "train": transforms.Compose([transforms.RandomResizedCrop(224),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]),
        "val": transforms.Compose([transforms.Resize((224, 224)),
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])}

    train_dataset = datasets.ImageFolder(root="/mnt/c/Users/santhosh/Downloads/MedMamba-main/MedMamba-main/colored_images",
                                         transform=data_transform["train"])
    train_num = len(train_dataset)

    flower_list = train_dataset.class_to_idx
    cla_dict = dict((val, key) for key, val in flower_list.items())
    # write dict into json file
    json_str = json.dumps(cla_dict, indent=4)
    with open('class_indices.json', 'w') as json_file:
        json_file.write(json_str)

    batch_size = 32
    nw = min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8])  # number of workers
    print('Using {} dataloader workers every process'.format(nw))

    train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size, shuffle=True,
                                               num_workers=nw)

    validate_dataset = datasets.ImageFolder(root="/mnt/c/Users/santhosh/Downloads/MedMamba-main/MedMamba-main/colored_images",
                                            transform=data_transform["val"])
    val_num = len(validate_dataset)
    validate_loader = torch.utils.data.DataLoader(validate_dataset,
                                                  batch_size=batch_size, shuffle=False,
                                                  num_workers=nw)
    print("using {} images for training, {} images for validation.".format(train_num,
                                                                           val_num))


    net = MedMamba.VSSM(num_classes=5)
    net.to(device)
    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=0.0001)

    epochs = 100
    best_acc = 0.0
    save_path = './{}Net.pth'.format("MedmambaNet1")
    train_steps = len(train_loader)
    for epoch in range(epochs):

        

        # train
        net.train()
        running_loss = 0.0
        train_bar = tqdm(train_loader, file=sys.stdout)
        for step, data in enumerate(train_bar):
            images, labels = data
            optimizer.zero_grad()
            outputs = net(images.to(device))
            loss = loss_function(outputs, labels.to(device))
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()

            train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch + 1,
                                                                     epochs,
                                                                     loss)

        # validate
        net.eval()
        acc = 0.0  # accumulate accurate number / epoch
        with torch.no_grad():
            val_bar = tqdm(validate_loader, file=sys.stdout)
            for val_data in val_bar:
                val_images, val_labels = val_data
                outputs = net(val_images.to(device))
                predict_y = torch.max(outputs, dim=1)[1]
                acc += torch.eq(predict_y, val_labels.to(device)).sum().item()

        val_accurate = acc / val_num
        print('[epoch %d] train_loss: %.3f  val_accuracy: %.3f' %
              (epoch + 1, running_loss / train_steps, val_accurate))

        patience=5
        if val_accurate > best_acc:
            best_acc = val_accurate
            torch.save(net.state_dict(), save_path)
            early_stop_counter = 0
        else:
            early_stop_counter += 1

        if early_stop_counter >= patience:
            print("Early stopping triggered. Stopping training.")
            break


    print('Finished Training')


if __name__ == '__main__':
    main()


torch.Size([1, 6])
using cuda:0 device.
Using 8 dataloader workers every process
using 3656 images for training, 3656 images for validation.
100%|██████████| 115/115 [00:25<00:00,  4.55it/s]
[epoch 1] train_loss: 1.151  val_accuracy: 0.633
100%|██████████| 115/115 [00:25<00:00,  4.53it/s]
[epoch 2] train_loss: 1.019  val_accuracy: 0.689
100%|██████████| 115/115 [00:25<00:00,  4.53it/s]
[epoch 3] train_loss: 0.976  val_accuracy: 0.711
100%|██████████| 115/115 [00:25<00:00,  4.51it/s]
[epoch 4] train_loss: 0.931  val_accuracy: 0.712
100%|██████████| 115/115 [00:25<00:00,  4.52it/s]
[epoch 5] train_loss: 0.892  val_accuracy: 0.679
100%|██████████| 115/115 [00:25<00:00,  4.54it/s]
[epoch 6] train_loss: 0.870  val_accuracy: 0.713
100%|██████████| 115/115 [00:25<00:00,  4.53it/s]
[epoch 7] train_loss: 0.845  val_accuracy: 0.699
100%|██████████| 115/115 [00:25<00:00,  4.52it/s]
[epoch 8] train_loss: 0.816  val_accuracy: 0.712
100%|██████████| 115/115 [00:25<00:00,  4.51it/s]
[epoch 9] train_l

In [27]:
import torch
from sklearn.metrics import precision_score, recall_score, f1_score
from torchvision import transforms, datasets
import MedMamba  # Import your model class

# Define device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load the saved model
model = MedMamba.VSSM(num_classes=5)  # Replace with your model class
model.load_state_dict(torch.load("/mnt/c/Users/santhosh/Downloads/MedMamba-main/MedMamba-main/MedMambagit/MedmambaNet1Net.pth"))
model.to(device)
model.eval()

from torchvision import transforms, datasets

data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load test dataset
test_dataset = datasets.ImageFolder(root="/mnt/c/Users/santhosh/Downloads/MedMamba-main/MedMamba-main/colored_images", transform=data_transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

# Get the number of classes
num_classes = len(test_dataset.classes)
class_names = test_dataset.classes


In [28]:
import numpy as np

# Initialize metrics
all_labels = []
all_preds = []
total_loss = 0.0
loss_function = torch.nn.CrossEntropyLoss()

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)

        # Compute loss
        loss = loss_function(outputs, labels)
        total_loss += loss.item()

        # Get predictions
        preds = torch.max(outputs, dim=1)[1]
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Convert to numpy arrays
all_labels = np.array(all_labels)
all_preds = np.array(all_preds)

# Compute accuracy
accuracy = np.mean(all_preds == all_labels)

# Compute precision, recall, F1-score for each class
precision = precision_score(all_labels, all_preds, average=None, labels=range(num_classes))
recall = recall_score(all_labels, all_preds, average=None, labels=range(num_classes))
f1 = f1_score(all_labels, all_preds, average=None, labels=range(num_classes))

# Compute overall metrics (macro-average)
precision_macro = precision_score(all_labels, all_preds, average='macro')
recall_macro = recall_score(all_labels, all_preds, average='macro')
f1_macro = f1_score(all_labels, all_preds, average='macro')

print(f"Test Loss: {total_loss / len(test_loader):.4f}")
print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Precision (per class): {precision}")
print(f"Recall (per class): {recall}")
print(f"F1-score (per class): {f1}")
print(f"Precision (macro): {precision_macro:.4f}")
print(f"Recall (macro): {recall_macro:.4f}")
print(f"F1-score (macro): {f1_macro:.4f}")


Test Loss: 0.4758
Test Accuracy: 83.29%
Precision (per class): [0.66666667 0.72226999 0.95018747 0.72897196 0.67088608]
Recall (per class): [0.60989011 0.84084084 0.98282548 0.52881356 0.2746114 ]
F1-score (per class): [0.63701578 0.77705828 0.96623094 0.6129666  0.38970588]
Precision (macro): 0.7478
Recall (macro): 0.6474
F1-score (macro): 0.6766
